# 예측 1

라이브러리

In [1]:
# 공통
%matplotlib inline
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 예측 1
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# 예측 2
from scipy.stats import norm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [19]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('전체.csv', encoding='cp949')
# data = data[38:]

# data = data.drop(['price'], axis = 1)
data = data.drop(['전체_temp'], axis = 1)
data = data.drop(['전체_hum'], axis = 1)

data = data.drop(['강원_temp'], axis = 1)
data = data.drop(['경기_temp'], axis = 1)
data = data.drop(['경남_temp'], axis = 1)
data = data.drop(['경북_temp'], axis = 1)
data = data.drop(['광주_temp'], axis = 1)
data = data.drop(['대구_temp'], axis = 1)
data = data.drop(['대전_temp'], axis = 1)
data = data.drop(['부산_temp'], axis = 1)
data = data.drop(['서울_temp'], axis = 1)
data = data.drop(['세종_temp'], axis = 1)
data = data.drop(['울산_temp'], axis = 1)
data = data.drop(['충남_temp'], axis = 1)
data = data.drop(['충북_temp'], axis = 1)
data = data.drop(['전남_temp'], axis = 1)
data = data.drop(['전북_temp'], axis = 1)
data = data.drop(['제주_temp'], axis = 1)
data = data.drop(['인천_temp'], axis = 1)

data = data.drop(['강원_hum'], axis = 1)
data = data.drop(['경기_hum'], axis = 1)
data = data.drop(['경남_hum'], axis = 1)
data = data.drop(['경북_hum'], axis = 1)
data = data.drop(['광주_hum'], axis = 1)
data = data.drop(['대구_hum'], axis = 1)
data = data.drop(['대전_hum'], axis = 1)
data = data.drop(['부산_hum'], axis = 1)
data = data.drop(['서울_hum'], axis = 1)
data = data.drop(['세종_hum'], axis = 1)
data = data.drop(['울산_hum'], axis = 1)
data = data.drop(['충남_hum'], axis = 1)
data = data.drop(['충북_hum'], axis = 1)
data = data.drop(['전남_hum'], axis = 1)
data = data.drop(['전북_hum'], axis = 1)
data = data.drop(['제주_hum'], axis = 1)
data = data.drop(['인천_hum'], axis = 1)
    
data = data.set_index('date')
data = data.fillna(method='ffill')
data.tail()

,price
date,
2021-12-27,48002138
2021-12-28,38070210
2021-12-29,52093193
2021-12-30,47463341
2021-12-31,66341333


In [20]:
# 인덱스 지정(?)

# strptime은 날짜와 시간 형식의 문자열을 datetime으로 변환해 줌
# strftime은 날짜와 시간(datetime)을 문자열로 출력
start = datetime.datetime.strptime("2021-01-01", "%Y-%m-%d")

# 기준 날짜에 대한 연산이 필요할 때 dateutil.relativedelta모듈의 relativedelta함수 이용
date_list = [start + relativedelta(days = x) for x in range(0, len(data)) ]

data['index'] = date_list

# set_index는 데이터프레임을 먼저 불러온 후, 특정 열을 인덱스로 설정할 때 사용
data.set_index( ['index'], inplace=True) 

# index 출력 X
data.index.name=None
data.head()

,price
2021-01-01,39209808
2021-01-02,62239672
2021-01-03,57365413
2021-01-04,95785399
2021-01-05,37415973


In [21]:
# coef값은 0에서 가장 떨어졌을때 가장 이상적
# p 값은 0과 가장 가까울때가 가장 좋습니다

mod = sm.tsa.statespace.SARIMAX(
    data['price'],
    order = (1, 1, 1),
    seasonal_order = (0, 1, 1, 12),
    # AR 매개 변수를 변환하여 모델의 자동 회귀 구성 요소에서 정상 성을 강제 적용할지 여부입니다. 기본값은 True입니다.
    # AR항이 stationary를 만족하게끔 강제하는 것
    enforce_stationarity = False,
    # 모델의 이동 평균 구성 요소에서 반전 성을 강제하기 위해 MA 매개 변수를 변환할지 여부입니다. 기본값은 True입니다.
    # MA항이 starionary를 만족하게끔 강제하는 것
    enforce_invertibility = False
)

results = mod.fit()

# print(results.summary())
print( results.summary().tables[1] )

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2255      0.125      1.797      0.072      -0.020       0.471
ma.L1         -0.8406      0.084     -9.979      0.000      -1.006      -0.675
ma.S.L12      -0.8969      0.054    -16.653      0.000      -1.002      -0.791
sigma2      1.035e+15   1.55e-17   6.68e+31      0.000    1.03e+15    1.03e+15


In [22]:
pred = results.get_prediction(
    start = pd.to_datetime('2021-01-01'),
    # 동적 회귀(?)
    dynamic = False
)

In [23]:
y_forecasted = pred.predicted_mean
y_truth = data['price']['2021-01-01' : ]

# Compute the Mean Square Error: 평균 제곱 오차 계산
mse = (( y_forecasted - y_truth ) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format( round(mse, 2) ) ) 

The Mean Squared Error of our forecasts is 469229974697253.8


In [24]:
# Get forecast some steps ahead in future 미래 예측
pred_uc = results.get_forecast( steps = 31 )

In [25]:
forecast = pred_uc.predicted_mean
forecast

2022-01-01    5.201499e+07
2022-01-02    5.080725e+07
2022-01-03    5.201617e+07
2022-01-04    5.724243e+07
2022-01-05    5.990946e+07
2022-01-06    5.762789e+07
2022-01-07    5.297533e+07
2022-01-08    5.302444e+07
2022-01-09    5.469327e+07
2022-01-10    5.583809e+07
2022-01-11    5.399650e+07
2022-01-12    5.466670e+07
2022-01-13    4.995291e+07
2022-01-14    5.091245e+07
2022-01-15    5.261002e+07
2022-01-16    5.794645e+07
2022-01-17    6.063832e+07
2022-01-18    5.836235e+07
2022-01-19    5.371106e+07
2022-01-20    5.376045e+07
2022-01-21    5.542935e+07
2022-01-22    5.657418e+07
2022-01-23    5.473258e+07
2022-01-24    5.540279e+07
2022-01-25    5.068900e+07
2022-01-26    5.164854e+07
2022-01-27    5.334611e+07
2022-01-28    5.868254e+07
2022-01-29    6.137441e+07
2022-01-30    5.909844e+07
2022-01-31    5.444715e+07
Freq: D, Name: predicted_mean, dtype: float64

In [26]:
# 예측 데이터 csv로 저장
forecast.to_csv('변수X_2022.csv', index=False)

PermissionError: [Errno 13] Permission denied: '변수X_2022.csv'

In [75]:
actual = [55461518159, 53165103786, 55913862375, 55660372637, 60330542263, 60734116670]

mse = mean_squared_error(actual, forecast)
rmse = np.sqrt(mse)
rmse

2526764852.398417